In [1]:
# Imports
import torch
import os 
import sys
from dask_jobqueue import SLURMCluster
from distributed import Client
from pathlib import Path

import do_ablation_nystrom as file

In [2]:
# Args setup
ntrial = 50
n = 100
ntest = 100
d = 5
noise = 0.5
features = 100
samples = 1000
calibrate = [False, True, True]
sample_split = [False, False, True]
marginal_loss = False
retrain_hypers = False
kernel = "gaussian"
approx = "mercer"

In [3]:
# Cluster creation
cluster = SLURMCluster(
    n_workers=0,
    memory="32GB",
    processes=1,
    cores=1,
    scheduler_options={
        "dashboard_address": ":11112",
        "allowed_failures": 1
    },
    job_cpu=1,
    walltime="3:0:0",
    job_extra_directives = ["-p medium,fast,cpu"],
)
cluster.adapt(minimum=0, maximum=200)
client = Client(cluster)

In [4]:
# Submitting jobs
futures = []
for seed in range(ntrial):
    for i in range(len(calibrate)):
            f = client.submit(file.main,seed,n,ntest,d,noise,
                              calibrate = calibrate[i],
                              sample_split = sample_split[i],
                              marginal_loss = marginal_loss,
                              retrain_hypers = retrain_hypers,
                              features = features, samples = samples,
                              approx = approx,
                              kernel = "gaussian")
            futures += [f]

In [5]:
# Check on futures
futures

[<Future: pending, key: main-ff561a2097e24f012f45d6b9dfd4616e>,
 <Future: pending, key: main-0896f52ed65651ce1178cd52ed57390b>,
 <Future: pending, key: main-6801f458b0a71e81677ffb55b0c3e09c>,
 <Future: pending, key: main-2eb3f5488ac23ba09e9b0d457a84719e>,
 <Future: pending, key: main-789a1891000433d2fbb72388e899cfb1>,
 <Future: pending, key: main-088927d4b57cfc3b6ae22379ce75bde2>,
 <Future: pending, key: main-de7f8f540a704a815b31d492828bada2>,
 <Future: pending, key: main-eb403e99d46f6154c750786ea409d267>,
 <Future: pending, key: main-3c5475db732a27e046a0e449a4de0191>,
 <Future: pending, key: main-d9bc39a4ccbd21b3bbe32bce57066c71>,
 <Future: pending, key: main-6f09e8d857d38fb8da5f81ae4380bb7c>,
 <Future: pending, key: main-b912e4517f0e28b03462b5fc44f8dbe6>,
 <Future: pending, key: main-c550725be4e2dcfe5e03dcc1d8d306e4>,
 <Future: pending, key: main-b8096a3ae3f07532b3b239c2085f5d90>,
 <Future: pending, key: main-7de531b330f07d0ab9ac7674655dc829>,
 <Future: pending, key: main-119ade88d9c

In [6]:
# Getting results
results = client.gather(futures)

In [7]:
# Closing client
client.close()
cluster.close()

In [8]:
# Saving results
torch.save(obj = results,
           f = "ablation_nystrom_ntrial={0}_n={1}_d={2}_noise={3}_new.pt".format(ntrial,n,d,noise))